In [122]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [123]:
### Function to Get All Tickers
def get_tickers(url):
    
    tickers_list = []
    # Get Tickers into df
    df = pd.read_html(url)
    # Convert the df to a list
    table = df[0]  # First table contains tickers
    tickers = table['Symbol'].tolist()
    # Yahoo Finance uses '-' instead of '.' (e.g., BRK.B → BRK-B)
    tickers_list = [ticker.replace('.', '-') for ticker in tickers]

    return tickers_list

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
all_tickers = get_tickers(url)
print(all_tickers)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', '

In [124]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['AAPL', 'MSFT']
start_date = '2025-01-01'
end_date = '2025-06-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_53809/17277343.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  2 of 2 completed

        Date        AAPL        MSFT
0 2025-01-02  243.263199  416.976868
1 2025-01-03  242.774368  421.728607
2 2025-01-06  244.410416  426.211365
3 2025-01-07  241.627136  420.752350
4 2025-01-08  242.115952  422.933960


In [125]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()
    
    # Drop null vals
    df.dropna(subset=['PCT ' + t for t in tickers], inplace=True)

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.head())


        Date        AAPL        MSFT  PCT AAPL  PCT MSFT
1 2025-01-03  242.774368  421.728607 -0.002009  0.011396
2 2025-01-06  244.410416  426.211365  0.006739  0.010629
3 2025-01-07  241.627136  420.752350 -0.011388 -0.012808
4 2025-01-08  242.115952  422.933960  0.002023  0.005185
5 2025-01-10  236.280045  417.345459 -0.024104 -0.013214


In [126]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        df['Signal ' + t] = (df['PCT ' + t] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{'Signal ' + t}: {df['Signal ' + t].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.05)
print(stock_data.head())


Signal AAPL: 2 buy signals generated
Signal MSFT: 1 buy signals generated
        Date        AAPL        MSFT  PCT AAPL  PCT MSFT  Signal AAPL  \
1 2025-01-03  242.774368  421.728607 -0.002009  0.011396            0   
2 2025-01-06  244.410416  426.211365  0.006739  0.010629            0   
3 2025-01-07  241.627136  420.752350 -0.011388 -0.012808            0   
4 2025-01-08  242.115952  422.933960  0.002023  0.005185            0   
5 2025-01-10  236.280045  417.345459 -0.024104 -0.013214            0   

   Signal MSFT  
1            0  
2            0  
3            0  
4            0  
5            0  


In [127]:
### Trade Simulation Function
''' 
Enter trade the day after the signal and exit after a fixed holding period (ex. 5 days)
'''
def trade_sim(ticker, df, holding_days):
    ### Empty List to Store Trades
    trades = []

    # Iterate over the df, stopping with the number of holding days left (avoids out of bounds error) 
    for i in range(len(df) - holding_days):
        # Identify the Buy Signals - Locate rows with a "1" indicator
        if df.iloc[i]['Signal ' + ticker] == 1:
            # Set Entry/Exit Date - For each signal date (i), set the entry date to be the next trading day (i + 1)
            entry_idx = i + 1
            exit_idx = i + holding_days

            entry_date = df.index[entry_idx]
            exit_date = df.index[exit_idx]

            # Record Entry and Exit Prices
            entry_price = df.iloc[entry_idx][ticker]
            exit_price = df.iloc[exit_idx][ticker]

            # Compute the Return for the Asset
            returns = (exit_price / entry_price) - 1

            # Save trade to list
            trades.append({
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Entry Price': entry_price,
                'Exit Price': exit_price,
                'Return': returns
            })

    return pd.DataFrame(trades, columns = ['Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Return'])

### Function to Return the Trade Simulation Function
#for t in tickers:
trades = trade_sim('AAPL', stock_data, holding_days = 14)
print(trades)


   Entry Date  Exit Date  Entry Price  Exit Price    Return
0          63         76   188.133301  208.097107  0.106115
1          64         77   181.222366  209.005920  0.153312


In [ ]:
### Function to Simulate all Trades
def simulate_all_trades(tickers, df, holding_days):
    all_trades = []

    for ticker in tickers:
        trades_df = trade_sim(ticker, df, holding_days)
        trades_df['Ticker'] = ticker  # Add column for traceability
        all_trades.append(trades_df)
    # Return the concatenated all trades df
    return pd.concat(all_trades, ignore_index = True)

### Function Return for all Tickers
all_trades = simulate_all_trades(all_tickers, stock_data, holding_days = 14)
all_trades.head()


KeyError: 'Signal MMM'

In [129]:
### Evaluate Total Performance
def performance_metrics(df, holding_days):
    # Total Return
    total_rets = (1 + df['Return']).prod() - 1
    # Avg Return
    avg_rets = df['Return'].mean()
    # Win Rate = Num Winning Trades / Total Trades
    win_rate = (df['Return'] > 0).mean()
    # Sharpe Ratio

    sharpe = (avg_rets / df['Return'].std()) * np.sqrt(252 / holding_days)

    print(f"Number of Trades: {len(df)}")
    print(f"Total Return: {total_rets:.2%}")
    print(f"Average Trade Return: {avg_rets:.2%}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")

### Function Return to Evaluate Total Performance
metrics = performance_metrics(trades, holding_days = 14)
print(metrics)


Number of Trades: 2
Total Return: 27.57%
Average Trade Return: 12.97%
Win Rate: 100.00%
Sharpe Ratio: 16.49
None
